## This script does an special type of dashboards to specific countries, if you want to add one here just add de country id to countries and then add the countri id and file to the countries_files dic. Be carefull to understand this type of dashboard, the most commun is the other one. 

In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import wabi
from wabi import sql_wabilytics,gSheets,maps_2b
from shapely.geometry import Point, Polygon
import datetime
from datetime import timedelta
import boto3
import io
from gspread_dataframe import set_with_dataframe
import sys
from shapely import wkt
import numpy as np
import time

C:\Users\camilafernandez_wabi\Documents\Notebooks


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wab

In [2]:
print('start scripting at '+str(datetime.datetime.utcnow())+'')

start scripting at 2022-12-07 13:22:37.451577


## Sheets and Countrys

In [3]:
countries = ['ke',
             'ma'
             #'eg'
            ]

In [4]:
#cambié la key de los los files para hacer las pruebillas
countries_files = {
    'ke': '1GHVsvDAIR3sY5ps4YbNilY_V-MJleLteG8-0LymHnkE',
    'ma':'1ED1amF3VN23LjZYh2kdMISTSPhD_ySsYdfAQRF006RM'
    #'eg' : '1UOWiyVf4fptD8cVhGRem0QtIypid5mzo8Y-zkppLfDY'
                  }
stores_paste_boolean = True
orders_paste_boolean = True
products_paste_boolean = True
metrics_paste_boolean = True

## Define Scope

In [5]:
#function to extract the week number
def week_number(date):    
    week = date.isocalendar()[1]
    return week

#function to extract the year
def extract_year(date):
    year = date.isocalendar()[0]
    return year

#function to extract the date of the start of the week
def GetStartDateFromWeek(year,week):
    iso_date = str(year)+'-W'+str(week)
    start_date_of_week = datetime.datetime.strptime(iso_date + '-1', "%Y-W%W-%w").date()
    return start_date_of_week

In [6]:
#La idea es tener 4 semanas para atrás y la semana en curso
tscope_weeks = 4
tscope_end = datetime.date.today()
tscope_end_str = tscope_end.strftime('%Y-%m-%d')

week = week_number(tscope_end)
year = extract_year(tscope_end)
iso_date = str(year)+'-W'+str(week)
start_date_of_week = GetStartDateFromWeek(year,week)
tscope_start = pd.Timestamp((start_date_of_week+timedelta(weeks=-(tscope_weeks))))
tscope_start_str = tscope_start.strftime('%Y-%m-%d')

In [7]:
weeks_list = []
i = 0
while i<=4:
    week_start = pd.Timestamp((start_date_of_week+timedelta(weeks=-(i))))
    week_start_str = week_start.strftime('%Y-%m-%d')
    weeks_list.append(week_start_str) 
    i+=1

## S3 imports

In [8]:
s3 = boto3.client('s3')
bucket0 = 'countries-states-2b'
obj = s3.get_object(Bucket=bucket0, Key='states0.csv')
states0 = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [9]:
countries_query = '''
SELECT 
    c.country_id,
    c.name as country
FROM wabi2bCross.country c
'''

In [10]:
 country0 = sql_wabilytics.querySQL(countries_query,schema = 'wabi2bCross')

In [11]:
bucket2 = 'wabi-ops-zones'
obj = s3.get_object(Bucket=bucket2, Key='wabi2b-zones/wabi2b_zones.kml')
full2b_zones_kml = gpd.GeoDataFrame.from_file(io.BytesIO(obj['Body'].read()))

#MLOLONGOOOOO
s3 = boto3.resource('s3')

obj = s3.Bucket('wabi-ops-zones').Object('wabi2b-zones/mlolongo_region.kml').get()
mlolongo_zones_kml= gpd.GeoDataFrame.from_file(obj['Body'], index_col=0)

In [12]:
#cities
full_zones = gpd.GeoDataFrame()
for ciudad in [full2b_zones_kml]:
    full_zones = full_zones.append(ciudad,ignore_index=True)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\2473404724.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_zones = full_zones.append(ciudad,ignore_index=True)


In [13]:
#MOLOLONGOOOO
ciudades_mp= [mlolongo_zones_kml]

zonas_mp = gpd.GeoDataFrame()
for ciudad in ciudades_mp:
    zonas_mlolongo=zonas_mp.append(ciudad,ignore_index=True)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\556247712.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas_mlolongo=zonas_mp.append(ciudad,ignore_index=True)


In [14]:
print(tscope_start_str)

2022-11-07


In [15]:
%%time
##Trae el cambio de moneda
exchanges = '''SELECT er.rate_date AS timestamp
, er.currency_rate AS currency
, er.rate 
, c.country_id 
FROM exchangeRate.exchange_rate AS er 
INNER JOIN wabi2b.country AS c
ON er.currency_rate = c.currency
WHERE er.rate_date >= "'''+str(tscope_start_str)+'''"'''
currency_df = sql_wabilytics.querySQL(exchanges)
currency_df['timestamp'] = pd.to_datetime(currency_df['timestamp']).dt.date

CPU times: total: 62.5 ms
Wall time: 3.6 s


## Status Func

In [16]:
def order_status(ts_confirmed,ts_shipped,ts_canceled):
    status = 'PENDING'
    if ts_confirmed == ts_confirmed:
        status = 'CONFIRMED'
    if ts_canceled == ts_canceled:
        status = 'CANCELED'
    if ts_shipped == ts_shipped:
        status = 'SHIPPED'
        
    return status

### Global data

In [17]:
bucket = 'wabi2b-evolution'
s3 = boto3.client('s3')

obj = s3.get_object(Bucket=bucket, Key='global_data.parquet')
global_data0 = pd.read_parquet(io.BytesIO(obj['Body'].read()))

In [18]:
global_data0_reduced = global_data0[['supplier_order_id','order_id','customerId','customer','supplier_id',
                            'supplier','created','confirmed_at','ship_at_date',
                            'canceled_by',
                            'total','discount','credits_paid','money_paid','lat','lon',
                            'country_id','customer_type','status','created_date','shippedAt']]

In [19]:
#acá filtro para quedarme solamente con los países que necesito
global_data0_reduced = global_data0_reduced.loc[global_data0_reduced['country_id'].isin(countries)].reset_index(drop=True)

In [20]:
global_data0_reduced = global_data0_reduced[(global_data0_reduced['created_date'] >=tscope_start_str) &
                                        (global_data0_reduced['created_date']<= tscope_end_str)]

In [21]:
#Me hago una lista de todas las suppliers_orders_id para después inyectarla en la query
sup_orders_id_list = global_data0_reduced['supplier_order_id'].tolist()

### Products Data

In [22]:
#aca voy a filtrar por las sup_orders_id (una vez que filtre arriba)
products_query = '''
SELECT
	soi.supplier_order_id,
	soi.product_id,
	soi.quantity,
	soi.units,
	soi.ean,
	soi.price,
	soi.name as product_name,
	soi.brand,
	soi.category,
	so.canceled_at,
	soi.amount as gmv,
	so.service_fee,
	so.delivery_cost,
	so.discount_id,
    so.final_amount,
	c.phone,
	d.code as discount_name,
	c.state_name as iso_code,
	so.formatted_address as delivery_address
FROM wabi2b.supplier_order so 
INNER JOIN wabi2b.supplier_order_items soi 
ON soi.supplier_order_id = so.supplier_order_id
INNER JOIN wabi2b.customers c 
ON c.customerId = so.customer_id 
LEFT JOIN wabi2b.discounts d 
ON d.discount_id = so.discount_id
where soi.supplier_order_id in '''+ wabi.stringify(sup_orders_id_list)


In [23]:
%%time
products_df_query = sql_wabilytics.querySQL(products_query,'wabi2b')

CPU times: total: 391 ms
Wall time: 1min 10s


In [24]:
products_df = products_df_query.copy(deep = True)

In [25]:
products_df.columns

Index(['supplier_order_id', 'product_id', 'quantity', 'units', 'ean', 'price',
       'product_name', 'brand', 'category', 'canceled_at', 'gmv',
       'service_fee', 'delivery_cost', 'discount_id', 'final_amount', 'phone',
       'discount_name', 'iso_code', 'delivery_address'],
      dtype='object')

In [26]:
orders_products_with_global_data = global_data0_reduced.merge(products_df, how='inner', on = 'supplier_order_id').reset_index(drop =True)

In [27]:
orders_products_with_global_data.columns

Index(['supplier_order_id', 'order_id', 'customerId', 'customer',
       'supplier_id', 'supplier', 'created', 'confirmed_at', 'ship_at_date',
       'canceled_by', 'total', 'discount', 'credits_paid', 'money_paid', 'lat',
       'lon', 'country_id', 'customer_type', 'status', 'created_date',
       'shippedAt', 'product_id', 'quantity', 'units', 'ean', 'price',
       'product_name', 'brand', 'category', 'canceled_at', 'gmv',
       'service_fee', 'delivery_cost', 'discount_id', 'final_amount', 'phone',
       'discount_name', 'iso_code', 'delivery_address'],
      dtype='object')

In [28]:
orders_products_with_global_data.rename(columns = {'customerId':'customer_id','customer':'customer_name',
                                                  'supplier':'supplier_name','created':'created_at',
                                                  'shippedAt':'shipped_at','ship_at_date':'ship_at',
                                                  'canceled_by':'cancelled_by','discount':'discount_amount',
                                                  },inplace = True)

In [29]:
orders_products_with_global_data.columns

Index(['supplier_order_id', 'order_id', 'customer_id', 'customer_name',
       'supplier_id', 'supplier_name', 'created_at', 'confirmed_at', 'ship_at',
       'cancelled_by', 'total', 'discount_amount', 'credits_paid',
       'money_paid', 'lat', 'lon', 'country_id', 'customer_type', 'status',
       'created_date', 'shipped_at', 'product_id', 'quantity', 'units', 'ean',
       'price', 'product_name', 'brand', 'category', 'canceled_at', 'gmv',
       'service_fee', 'delivery_cost', 'discount_id', 'final_amount', 'phone',
       'discount_name', 'iso_code', 'delivery_address'],
      dtype='object')

## Separate The Raw Data in the Orders and the Products DF

In [30]:
orders0 = orders_products_with_global_data[['supplier_order_id', 'order_id','customer_id', 'customer_name', 'supplier_id',
                            'supplier_name', 'created_at' ,'status', 'confirmed_at' , 'canceled_at',
                            'ship_at', 'cancelled_by', 'total',
                            'discount_amount', 'credits_paid' , 'money_paid' , 'service_fee' ,'lat','lon',
                            'delivery_cost','final_amount','country_id','phone','customer_type',
                            'discount_id','discount_name','iso_code','shipped_at']]

In [31]:
print('before: ', orders0.shape[0])
orders0 = orders0.drop_duplicates().reset_index(drop=True)
print('after: ', orders0.shape[0])

before:  29164
after:  17032


In [32]:
products0 = orders_products_with_global_data[['supplier_order_id', 'order_id', 'supplier_id','supplier_name', 'created_at',
                              'status', 'lat', 'lon','total' ,"product_id", 'ean' ,'product_name','brand','category',
                              'units', 'price','quantity', 'gmv', 'discount_amount', 'country_id' ,
                              'iso_code']]

## Generate the Variables for the orders DataFrames

In [33]:
#zonify takes a while so it is better to do the split here (kenya's df gets filtered so it is less data)
countries_orders = {}

for c in countries:
    countries_orders[c] = c+'_orders'

print(countries_orders)

for c in countries_orders:
    vars()[countries_orders[c]] = orders0[orders0['country_id'] == c].reset_index(drop=True)

{'ke': 'ke_orders', 'ma': 'ma_orders'}



## Zonify

In [34]:
for c in countries_orders:
    vars()[countries_orders[c]] = maps_2b.zonify(
        vars()[countries_orders[c]].reset_index(drop=True),
        zonas=full_zones,column='city').reset_index(drop=True)
    
    #vars()[countries_orders[c]].rename(columns= {'zona':'city'}, inplace=True)
    vars()[countries_orders[c]] = vars()[countries_orders[c]].drop(columns={'geometry'})
    
    vars()[countries_orders[c]] = vars()[countries_orders[c]].merge(
        states0,how='left',on='iso_code').drop(columns={'iso_code'})
    
    vars()[countries_orders[c]] = maps_2b.zonify(
        vars()[countries_orders[c]],
        maps_2b.zonas,column='zone').reset_index(drop=True)
    if 'ke' in c:
        
        vars()[countries_orders[c]] = maps_2b.zonify(
            vars()[countries_orders[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
    
    vars()[countries_orders[c]] = vars()[countries_orders[c]].drop(columns={'geometry'})

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:02.355070


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

...

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:01:21.662511


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.628130
Zonify time: 0:00:00.117929
Zonify time: 0:00:07.915147


### Seguimo con Customers

In [35]:
%%time
buffer = io.BytesIO()
s3 = boto3.resource('s3')
object = s3.Object('wabi2b-evolution','customers_evo.parquet')
object.download_fileobj(buffer)
customers1 = pd.read_parquet(buffer)

CPU times: total: 2.81 s
Wall time: 13.5 s


In [36]:
customers1['country_id'].unique()

array(['vn', 'in', 'ng', 'ke', 'my', 'mx', 'eg', 'ph', 'pt', 'ma', 'us',
       'th'], dtype=object)

In [37]:
customers1=customers1[customers1['country_id'].isin(countries)].reset_index(drop=True)

In [38]:
countries

['ke', 'ma']

In [39]:
customers1 = maps_2b.zonify(customers1.reset_index(drop=True) ,zonas=full_zones).reset_index(drop=True)
customers1.rename(columns= {'zona':'city'}, inplace=True)
customers1 = customers1.drop(columns={'geometry'})

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:02.713422


In [40]:
customers2 = customers1.merge(states0,how='left',on='iso_code').drop(columns={'iso_code'})

In [41]:
customers3 = maps_2b.zonify(customers2,maps_2b.zonas,column = 'zone')
customers3 = customers3.drop(columns={'geometry'}).reset_index(drop=True)

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

...

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:01:59.971085


In [42]:
customers3 = customers3.drop(columns={
    'owner_id'
    , 'hunter_contact_data'
    ,'first_name'
    ,'last_name'
})

### Generate the variables for the customers dataframes

In [43]:
countries_customers = {}

for c in countries:
    countries_customers[c] = c+'_customers'

print(countries_customers)

for c in countries_customers:
    vars()[countries_customers[c]] = customers3[customers3['country_id'] == c].reset_index(drop=True)

{'ke': 'ke_customers', 'ma': 'ma_customers'}


In [44]:
for c in countries_customers:
    if 'ke' in c:       
        vars()[countries_customers[c]] = maps_2b.zonify(
            vars()[countries_customers[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
        
        vars()[countries_customers[c]] = vars()[countries_customers[c]].drop(columns={'geometry'})
    else:
        pass

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


Zonify time: 0:00:00.892149


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


In [45]:
for c in countries_customers:
    vars()[countries_customers[c]] = vars()[countries_customers[c]].drop(columns={'additional_info'})

#### Customers Buyers

In [46]:
#look_countrys_customers=pd.concat([ke_customers, eg_customers], axis=0)
look_countrys_customers=pd.concat([ke_customers], axis=0)
customers_ids_lists=list(look_countrys_customers["customer_id"].astype(str))

In [47]:
vars()[countries_customers['ke']]

,country_id,customer_id,name,created_at,legal_id,phone,lat,lon,formatted_address,customer_status,cutomer_type,creation_type,created_by,city,state_name,zone,zone_mlolongo
0,ke,353288,Kimathi store,2022-06-10 12:54:16,27394500,2540726138665,-4.037000,39.624533,"XJ7F+6V, Mombasa, Kenya",APPROVED,Kiosks,bySALESREP,"Edwin Aludira, Musungu",Mombasa,Mombasa,Migadini,otro
1,ke,152258,Best price,2021-12-15 06:06:58,A99286747L,2540799286747,-1.249961,36.939538,"QW2Q+2R8, New Njiru Town, Kenya",APPROVED,Minimart,bySALESREP,"Kuria, Dan",Nairobi,Nairobi City,NJIRU,otro
2,ke,223253,Karasinka bakers pastries,2022-02-02 05:31:25,34679653,2540714352296,-1.396767,36.758024,"JQ35+759, Ongata Rongai, Kenya",APPROVED,Kiosks,bySALESREP,"Begi, Emmah",Nairobi,Nairobi City,RONGAI UPPER,otro
3,ke,199464,Walibora shop,2022-01-12 07:23:06,20765467,2540770760532,-1.261791,36.918893,"Near Kayole One Stage, Off Kayole Spine, Nairo...",REJECTED,Kiosks,bySALESREP,"Achieng, Pauline",Nairobi,Nairobi City,UMOJA 2/3,otro
4,ke,230206,Umoja store kwa bob,2022-02-07 10:35:54,2000416,2540712000416,-1.459544,36.976427,"GXRG+5G5, Athi River, Kenya",APPROVED,Kiosks,bySALESREP,"Wanjiru, Ann",Nairobi,Machakos,ATHIRIVER,otro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25699,ke,251488,Maguna Andu cereals shop,2022-02-21 11:35:57,321278955,2540713945113,-1.395657,36.738206,"JP3Q+P7M, Magadi Rd, Ongata Rongai, Kenya",APPROVED,Kiosks,bySALESREP,"Mumbi, Ann",Nairobi,Kajiado,RONGAI SOUTH,otro
25700,ke,129357,Super Duka 2,2021-11-30 10:20:26,A23446784A,2540776031939,-1.270153,36.913900,"Kayole, Nairobi, Kenya",REJECTED,Kiosks,bySALESREP,"Achieng, Pauline",Nairobi,Nairobi City,KAYOLE,otro
25701,ke,391919,Masai shop,2022-08-24 10:53:00,29281044,2540718543644,-1.322204,36.895208,"Shop 6, Ground Floor, Sabrina Building, Outer ...",PENDING,Orgánic,bySALESREP,"None, None",Nairobi,Nairobi City,PIPELINE,otro
25702,ke,291846,Jolly mali mali- 46,2022-04-02 11:49:12,29536501,2540710979382,-1.286879,36.747194,"46 Kawangware Rd, Nairobi, Kenya",APPROVED,Kiosks,bySALESREP,"Kirui, Ronald",Nairobi,Nairobi City,56,otro


In [48]:
%%time
customerss_bys = '''select
customer_id
, created_at
, confirmed_at
, shipped_at
, canceled_at
, supplier_order_id
, order_id
from  wabi2bOrders.supplier_order AS pd
where customer_id in '''+wabi.stringify(customers_ids_lists)+'''
and created_at between "2021-11-01 00:00:00"  and "'''+tscope_end_str+''' 23:59:59" 
group by customer_id'''
customerss_buys = sql_wabilytics.querySQL(customerss_bys,schema = 'wabi2bOrders')

CPU times: total: 219 ms
Wall time: 35.2 s


In [49]:
customerss_buys

,customer_id,created_at,confirmed_at,shipped_at,canceled_at,supplier_order_id,order_id
0,100004,2022-04-01 13:00:32,2022-04-01 13:47:49,2022-04-01 16:02:40,NaT,1212266,1111554
1,100012,2022-11-11 09:00:12,2022-11-11 12:43:43,2022-11-16 02:01:55,NaT,2620528,2431410
2,100030,2022-08-05 09:26:37,2022-08-05 15:14:22,2022-08-05 16:43:31,NaT,2058573,1904456
3,100047,2021-11-04 13:07:52,NaT,NaT,2021-11-04 15:34:40,471662,434707
4,100076,2021-11-04 13:17:23,2021-11-11 16:12:24,2021-11-11 16:13:27,NaT,471702,434745
...,...,...,...,...,...,...,...
16414,99962,2022-01-10 14:04:13,2022-01-11 05:18:40,2022-01-11 05:21:39,NaT,767153,707553
16415,99968,2021-11-04 12:59:15,2021-11-06 05:40:36,2021-11-06 16:24:23,NaT,471639,434685
16416,99970,2021-12-10 06:46:31,2021-12-10 11:51:46,2021-12-10 16:39:20,NaT,621620,573535
16417,99979,2021-12-22 09:27:14,2022-01-01 04:31:19,2022-01-01 04:31:23,NaT,681647,628837


In [50]:
customerss_buys['status'] = customerss_buys.apply(lambda x: order_status(x['confirmed_at'],x['shipped_at'],x['canceled_at']),axis=1)
customerss_buys_delivered=customerss_buys[customerss_buys["status"]=="SHIPPED"].reset_index(drop=True)
customers_total_orders_by_stores=customerss_buys.groupby(["customer_id"]).agg(
    total_orders=pd.NamedAgg(column="order_id", aggfunc="nunique")
).reset_index()
customers_delivered_orders_by_stores=customerss_buys_delivered.groupby(["customer_id"]).agg(
    total_delivered_orders=pd.NamedAgg(column="order_id", aggfunc="nunique")
).reset_index()
customers_total_orders_by_stores=customers_total_orders_by_stores.merge(customers_delivered_orders_by_stores, on="customer_id", how="left")
customers_total_orders_by_stores['total_orders'] = customers_total_orders_by_stores['total_orders'].fillna(0).astype(int)
customers_total_orders_by_stores['total_delivered_orders'] = customers_total_orders_by_stores['total_delivered_orders'].fillna(0).astype(int)

In [51]:
customers_total_orders_by_stores.rename(columns = {'customerId':'customer_id'},inplace = True)

In [52]:
for c in countries_customers:
    if 'ke' in c:
        vars()[countries_customers[c]] = vars()[countries_customers[c]].merge(customers_total_orders_by_stores, on="customer_id", how="left")
        vars()[countries_customers[c]]['total_orders']=vars()[countries_customers[c]]['total_orders'].fillna(0).astype(int)
        vars()[countries_customers[c]]['total_orders']=vars()[countries_customers[c]]['total_delivered_orders'].fillna(0).astype(int)
    elif 'eg' in c:
        vars()[countries_customers[c]] = vars()[countries_customers[c]].merge(customers_total_orders_by_stores, on="customer_id", how="left")
        vars()[countries_customers[c]]['total_orders']=vars()[countries_customers[c]]['total_orders'].fillna(0).astype(int)
        vars()[countries_customers[c]]['total_orders']=vars()[countries_customers[c]]['total_delivered_orders'].fillna(0).astype(int)
    else:
        pass

## Products

In [53]:
zonificable_df = products0[['lat','lon']]

In [54]:
#dropeamos los duplicados para que la zonificación sea mas eficiente
print('before: ', zonificable_df.shape[0])
zonificable_df = zonificable_df.drop_duplicates().reset_index(drop=True)
print('after: ', zonificable_df.shape[0])

before:  29164
after:  2527


In [55]:
zonify_df = maps_2b.zonify(zonificable_df.reset_index(drop=True) ,zonas=full_zones).reset_index(drop=True)

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.535168


In [56]:
zonify_df.rename(columns= {'zona':'city'}, inplace=True)
zonify_df = zonify_df.drop(columns={'geometry'})

In [57]:
products1 = products0.merge(zonify_df, how = 'left', on = ['lat','lon']).reset_index(drop = True)

In [58]:
products1 = products1.merge(states0,how='left',on='iso_code').drop(columns={'iso_code'})

In [59]:
products1['discount_percentage'] = products1['discount_amount'] / products1['total']
products1['discount_percentage'] = round(products1['discount_percentage'].astype(float),4)
products1.discount_percentage = products1.discount_percentage.fillna(0)

In [60]:
products1['discount_per_product'] = products1['gmv'] * products1['discount_percentage']
products1.discount_per_product = products1.discount_per_product.fillna(0)
products1['discount_per_product'] = round(products1['discount_per_product'].astype(float),2)

In [61]:
products1['timestamp'] = pd.to_datetime(products1['created_at']).dt.date

In [62]:
#merge the currency df to principal df
products1 = products1.merge(currency_df, how = 'left' , left_on = ['country_id','timestamp'] , right_on = ['country_id','timestamp'])
#column timestamp duplicated, droping the column
products1 = products1.drop(columns = 'timestamp')

In [63]:
products1['gmv_usd'] = round(products1['gmv'] / products1['rate'],2)

products1['discount_amount'] = products1['discount_amount'].astype(float)
products1['discount_amount_usd'] = round(products1['discount_amount'] / products1['rate'],2)

In [64]:
products2 = products1[['created_at','supplier_order_id','order_id','supplier_id','supplier_name','country_id','city','state_name',"product_id",
                       'currency','rate', 'ean','product_name','brand','category',
                       'units','price','quantity','gmv','gmv_usd','discount_amount',
                       'discount_percentage','discount_per_product']]
products2=products2.drop("product_id", axis=1)

In [65]:
countries_products = {}
for c in countries:
    countries_products[c] = c+'_products'
print(countries_products)
for c in countries_products:
        vars()[countries_products[c]] = products2[products2['country_id'] == c].reset_index(drop=True)

{'ke': 'ke_products', 'ma': 'ma_products'}


## Metrics

Agrupado por semana, por zona, por supplier

- Week
- country
- city
- zone
- supplier
- GMV
- discounts
- disc/gmv
- orders
- canceled orders
- delivered orders
- pending/confirmed orders
- Stores with purchase
- Registered stores
- New registered stores
- avg ticket
- lead time
- orders without coupons
- orders with coupons
- suppliers available 
- products available
- registered_stores
- new registers

In [66]:
countries_metrics_1 = {}
countries_metrics_2 = {}
for c in countries:
    countries_metrics_1[c] = c+'_metrics_1'
    countries_metrics_2[c] = c+'_metrics_2'
print(countries_metrics_1,'\n\n',countries_metrics_2)
#Esto lo hacemos para que mas abajo no se rompa todo si hay un df con un country_id que no tiene data
for c in countries_metrics_1.copy():
    vars()[countries_metrics_1[c]] = vars()[countries_orders[c]].copy(deep=True)


{'ke': 'ke_metrics_1', 'ma': 'ma_metrics_1'} 

 {'ke': 'ke_metrics_2', 'ma': 'ma_metrics_2'}


In [67]:
#map the function to the dataframes
#agrego el if porque si alguno de los dfs no tiene rows, rompe en la instrucción de crear la col 'week_start'
for c in countries_metrics_1:
    vars()[countries_metrics_1[c]]['year'] = vars()[countries_metrics_1[c]]['created_at'].map(extract_year)
    vars()[countries_metrics_1[c]]['week'] = vars()[countries_metrics_1[c]]['created_at'].map(week_number)
    vars()[countries_metrics_1[c]]['month'] = vars()[countries_metrics_1[c]]['created_at'].apply(
        lambda x: (pd.Timestamp(x)).date().strftime("%m"))
    if(len(vars()[countries_metrics_1[c]])>0):
        vars()[countries_metrics_1[c]]['week_start'] = vars()[countries_metrics_1[c]].apply(
            lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)
    else:
        vars()[countries_metrics_1[c]]['week_start'] =''

In [68]:
#create one hot encoding with the status column
for c in countries_metrics_1:
    #rompe esto con kenya
    vars()[countries_metrics_1[c]] = pd.get_dummies(vars()[countries_metrics_1[c]],columns=['status'])
    
    if 'PENDING' not in vars()[countries_orders[c]].status.unique():
        vars()[countries_metrics_1[c]]['status_PENDING'] = 0

    #calculate a new column with the lead time in hours between created_at and shipped_at
    vars()[countries_metrics_1[c]]['lead_time'] = (vars()[countries_metrics_1[c]]['shipped_at'] - vars()[countries_metrics_1[c]]['created_at']).dt.total_seconds()/3600
    vars()[countries_metrics_1[c]]['lead_time'].fillna(0, inplace=True)

#### A little break for save the orders to process later for KO/NO - KO Group Metrics Asia Africa

In [69]:
countries_KO_metrics = {}
for c in countries:
    countries_KO_metrics[c] = c+'_KO_metrics'

In [70]:
#tengo los paises separados en diferentes dfs
for c in countries_KO_metrics:
    print(c)
    vars()[countries_KO_metrics[c]] = vars()[countries_metrics_1[c]].copy(deep=True)

ke
ma


#### End of little break

In [71]:
for c in countries:
    vars()[countries_metrics_1[c]].total = vars()[countries_metrics_1[c]].total.astype(float)
    vars()[countries_metrics_2[c]] = vars()[countries_metrics_1[c]].copy(deep=True)

In [72]:
for c in countries_metrics_1:
    print(c)
    #the difference is only that the other countries do not have the special zone_mlolongo column
    if c == "ke":
        vars()[countries_metrics_1[c]] = vars()[countries_metrics_1[c]].groupby(['year','month','week','week_start','country_id','city','state_name','zone','zone_mlolongo','supplier_name','supplier_id']).agg(
            gmv = pd.NamedAgg(column='total' , aggfunc='sum'),
            discounts = pd.NamedAgg(column='discount_amount' , aggfunc='sum'),
            #agregar despues discounts/gmv cuando ya tenga el df agrupado
            orders = pd.NamedAgg(column='order_id' , aggfunc='nunique'),    
            supplier_orders = pd.NamedAgg(column='supplier_order_id' , aggfunc='nunique'),
            canceled_supplier_orders = pd.NamedAgg(column='canceled_at' , aggfunc='count'),
            deliv_supplier_orders = pd.NamedAgg(column='shipped_at' , aggfunc='count')
            ,pending_orders = pd.NamedAgg(column='status_PENDING' , aggfunc='sum')
            ,stores_w_purchase = pd.NamedAgg(column='customer_id' , aggfunc='nunique')
            ,avg_ticket = pd.NamedAgg(column='total' , aggfunc='mean')
            ,avg_lead_time = pd.NamedAgg(column='lead_time' , aggfunc='mean')
            #agregar despues orders_no_coupon como la resta entre supplier_orders menos orders_w_coupon
            ,orders_w_coupon = pd.NamedAgg(column='discount_id' , aggfunc='nunique')
            ).reset_index()  
        
    else:
        vars()[countries_metrics_1[c]] = vars()[countries_metrics_1[c]].groupby(['year','month','week','week_start','country_id','city','state_name','zone','supplier_name','supplier_id']).agg(
                gmv = pd.NamedAgg(column='total' , aggfunc='sum'),
                discounts = pd.NamedAgg(column='discount_amount' , aggfunc='sum'),
                #agregar despues discounts/gmv cuando ya tenga el df agrupado
                orders = pd.NamedAgg(column='order_id' , aggfunc='nunique'),    
                supplier_orders = pd.NamedAgg(column='supplier_order_id' , aggfunc='nunique'),
                canceled_supplier_orders = pd.NamedAgg(column='canceled_at' , aggfunc='count'),
                deliv_supplier_orders = pd.NamedAgg(column='shipped_at' , aggfunc='count')
                ,pending_orders = pd.NamedAgg(column='status_PENDING' , aggfunc='sum')
                ,stores_w_purchase = pd.NamedAgg(column='customer_id' , aggfunc='nunique')
                ,avg_ticket = pd.NamedAgg(column='total' , aggfunc='mean')
                ,avg_lead_time = pd.NamedAgg(column='lead_time' , aggfunc='mean')
                #agregar despues orders_no_coupon como la resta entre supplier_orders menos orders_w_coupon
                ,orders_w_coupon = pd.NamedAgg(column='discount_id' , aggfunc='nunique')
                ).reset_index()  
        #create a new column with the discount/gmv percentage
        vars()[countries_metrics_1[c]]['discount/gmv'] = vars()[countries_metrics_1[c]]['discounts'].astype(float)/vars()[countries_metrics_1[c]]['gmv'].astype(float)

        #create a new column with the orders without coupons (supplier_orders - orders_w_coupon)
        vars()[countries_metrics_1[c]]['orders_no_coupon'] = vars()[countries_metrics_1[c]]['supplier_orders'] - vars()[countries_metrics_1[c]]['orders_w_coupon']

ke
ma


### Group Metrics Asia-Africa KO/NO-KO
##### Una orden la consideramos KO si tiene al menos un producto KO

In [75]:
#definimos para que países queremos estas métricas, asi no procesamos data de mas
#group_metrics_countries = ['ke','eg']
group_metrics_countries = ['ke','ma']

In [76]:
#acá me voy guardando las sup orders id que tengo de cada país para después traerme los items 
#de esas ordenes 
sup_orders_id_list_aux = []
for c in group_metrics_countries:
    #acá tengo una lista de listas
    sup_orders_id_list_aux.append(list(vars()[countries_KO_metrics[c]]['supplier_order_id'].astype(str)))
    

In [77]:
sup_orders_id_list = []
for sup_list in sup_orders_id_list_aux:
    #concateno las dos listas para quedarme con una
    sup_orders_id_list += sup_list

In [78]:
supplier_orders_items_query = '''
SELECT
	soi.supplier_order_id,
	s.ko_tag,
    soi.amount,
    s.country_id,
    s.supplier_id,
    s.name              as supplier,
    soi.manufacturer,
    so.order_id,
    date(so.created_at) as created_at
FROM wabi2bOrders.supplier_order_items soi
LEFT JOIN wabi2bSuppliers.suppliers s
ON soi.supplier_id = s.supplier_id 
left join wabi2bOrders.supplier_order so 
on so.supplier_order_id = soi.supplier_order_id 
WHERE soi.supplier_order_id IN '''+wabi.stringify(sup_orders_id_list)+''';
'''

In [79]:
supplier_order_items_df_query = sql_wabilytics.querySQL(supplier_orders_items_query,'wabi2bOrders')

In [80]:
sup_orders_items_df = supplier_order_items_df_query.copy(deep = True)

In [81]:
currency_exchange = currency_df.copy(deep = True)

In [82]:
currency_exchange = currency_exchange[(currency_exchange['country_id']=='ke') | (currency_exchange['country_id']=='eg')]

In [83]:
currency_exchange['date'] = currency_exchange.apply(lambda x: x['timestamp'].strftime('%Y-%m-%d'),axis = 1)

In [84]:
for c in group_metrics_countries:
    if(len(vars()[countries_KO_metrics[c]]) > 0):
        vars()[countries_KO_metrics[c]]['date'] = vars()[countries_KO_metrics[c]].apply(lambda x: x['created_at'].strftime('%Y-%m-%d'),axis = 1)
    else:
        vars()[countries_KO_metrics[c]]['date'] = ''
    vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].merge(currency_exchange[['date','rate','country_id']], how = 'left', on = ['date','country_id'])   

In [85]:
#calcular el amount en usd para poder calcular el GMV usd después
for c in group_metrics_countries:
    if(len(vars()[countries_KO_metrics[c]])>0):
        vars()[countries_KO_metrics[c]]['usd_total'] = vars()[countries_KO_metrics[c]]['total']/vars()[countries_KO_metrics[c]]['rate']
    else:
        vars()[countries_KO_metrics[c]]['usd_total'] = ''

In [86]:
sup_orders_items_df["manufacturer"] = sup_orders_items_df["manufacturer"].apply(lambda x: x.lower())

In [87]:
#aplicamos el set primero para borrar duplicados
manufacturers_list = list(set(sup_orders_items_df["manufacturer"].astype(str)))

In [88]:
#Creamos una nueva lista y nos quedamos con los manufacturers que son de Coca - Cola
coca_manufacturer_list = []
for manufacturer in manufacturers_list:
    if manufacturer.find('coca')!=-1 or manufacturer.find('coke')!=-1:
        coca_manufacturer_list.append(manufacturer)

In [89]:
#flagueamos las ordenes que tienen productos de Coca - Cola
sup_orders_items_df['KO_products'] = sup_orders_items_df.apply(lambda x: 1 if x['manufacturer'] in coca_manufacturer_list else 0,axis = 1)

In [90]:
sup_orders_items_KO_df = sup_orders_items_df[sup_orders_items_df['KO_products']==1]

In [91]:
#dividimos las ordenes porque kenya tiene otro campo y calculamos las métricas diferente, ya tenemos las que son de coca
sup_orders_items = {}
for c in group_metrics_countries:
    sup_orders_items[c] = c+'_sup_orders_items'

for c in group_metrics_countries:
    vars()[sup_orders_items[c]] = sup_orders_items_KO_df[sup_orders_items_KO_df['country_id'] == c].reset_index(drop=True)

In [92]:
#esto lo hago para poder tener toda la info para agrupar y no volver a zonificar que es menos performante
for c in group_metrics_countries:
    if c == 'ke':
        vars()[sup_orders_items[c]] = vars()[sup_orders_items[c]].merge(
            vars()[countries_KO_metrics[c]][['supplier_order_id','state_name','city','zone','zone_mlolongo',
                                             'year','month','week','week_start','rate']], how = 'inner', on = 'supplier_order_id')
    else:
        vars()[sup_orders_items[c]] = vars()[sup_orders_items[c]].merge(
            vars()[countries_KO_metrics[c]][['supplier_order_id','state_name','city','zone',
                                             'year','month','week','week_start','rate']], how = 'inner', on = 'supplier_order_id')

In [93]:
#calcular el amount en usd para poder calcular el GMV usd después
for c in group_metrics_countries:
    vars()[sup_orders_items[c]]['usd_amount'] = vars()[sup_orders_items[c]]['amount']/vars()[sup_orders_items[c]]['rate']

In [94]:
for c in group_metrics_countries:
    if c== 'ke':
        #the difference is only that the other countries do not have the special zone_mlolongo column
        vars()[sup_orders_items[c]] = vars()[sup_orders_items[c]].groupby(['country_id','state_name','city','zone','zone_mlolongo',
                                                        'year','month','supplier','supplier_id','week','week_start',
                                                        'KO_products']).agg(
        gmv_ko = pd.NamedAgg(column='amount', aggfunc='sum'),
        gmv_ko_usd = pd.NamedAgg(column='usd_amount', aggfunc='sum'),
        orders_ko = pd.NamedAgg(column='order_id' , aggfunc='nunique')).reset_index()
    else:
        vars()[sup_orders_items[c]] = vars()[sup_orders_items[c]].groupby(['country_id','state_name','city','zone',
                                                        'year','month','supplier','supplier_id','week','week_start',
                                                        'KO_products']).agg(
        gmv_ko = pd.NamedAgg(column='amount', aggfunc='sum'),
        gmv_ko_usd = pd.NamedAgg(column='usd_amount', aggfunc='sum'),
        orders_ko = pd.NamedAgg(column='order_id' , aggfunc='nunique')).reset_index()

In [95]:
#acá nos quedamos con las sup orders id que tengan algún producto de KO para poder identificarlas abajo y
#no contar orders repetidas
sup_orders_id_KO_list = list(set(sup_orders_items_KO_df["supplier_order_id"].astype(str)))

In [96]:
for c in group_metrics_countries:
    vars()[countries_KO_metrics[c]].rename(columns= {'supplier_name':'supplier'}, inplace=True)
    if len(vars()[countries_KO_metrics[c]]) > 0:
        vars()[countries_KO_metrics[c]]['KO_products'] = vars()[countries_KO_metrics[c]].apply(lambda x: 1 if x['supplier_order_id'] in sup_orders_id_KO_list else 0,axis = 1)
    else:
        vars()[countries_KO_metrics[c]]['KO_products'] = ""
    vars()[countries_KO_metrics[c]].total = vars()[countries_KO_metrics[c]].total.astype(float)

In [97]:
for c in group_metrics_countries:
    if c== 'ke':
        #the difference is only that the other countries do not have the special zone_mlolongo column
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].groupby(['country_id','state_name','city','zone','zone_mlolongo',
                                                        'year','month','supplier','supplier_id','week','week_start',
                                                        'KO_products']).agg(
        gmv = pd.NamedAgg(column='total' , aggfunc='sum'),
        gmv_usd = pd.NamedAgg(column='usd_total' , aggfunc='sum'),
        discounts = pd.NamedAgg(column='discount_amount' , aggfunc='sum'),
        orders = pd.NamedAgg(column='order_id' , aggfunc='nunique'),
        supplier_orders = pd.NamedAgg(column='supplier_order_id' , aggfunc='nunique'),
        canceled_supplier_orders = pd.NamedAgg(column='canceled_at' , aggfunc='count'),
        deliv_supplier_orders = pd.NamedAgg(column='shipped_at' , aggfunc='count'),
        pending_orders = pd.NamedAgg(column='status_PENDING' , aggfunc='sum'),
        stores_w_purchase = pd.NamedAgg(column='customer_id' , aggfunc='nunique'),
        avg_ticket = pd.NamedAgg(column='total' , aggfunc='mean'),
        avg_lead_time = pd.NamedAgg(column='lead_time' , aggfunc='mean'),
        orders_w_coupon = pd.NamedAgg(column='discount_id' , aggfunc='nunique'),
        suppliers_available = pd.NamedAgg(column='supplier_id',aggfunc='nunique')
        ).reset_index()
    else:
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].groupby(['country_id','state_name','city','zone',
                                                        'year','month','supplier','supplier_id','week','week_start',
                                                        'KO_products']).agg(
        gmv = pd.NamedAgg(column='total' , aggfunc='sum'),
        gmv_usd = pd.NamedAgg(column='usd_total' , aggfunc='sum'),
        discounts = pd.NamedAgg(column='discount_amount' , aggfunc='sum'),
        orders = pd.NamedAgg(column='order_id' , aggfunc='nunique'),
        supplier_orders = pd.NamedAgg(column='supplier_order_id' , aggfunc='nunique'),
        canceled_supplier_orders = pd.NamedAgg(column='canceled_at' , aggfunc='count'),
        deliv_supplier_orders = pd.NamedAgg(column='shipped_at' , aggfunc='count'),
        pending_orders = pd.NamedAgg(column='status_PENDING' , aggfunc='sum'),
        stores_w_purchase = pd.NamedAgg(column='customer_id' , aggfunc='nunique'),
        avg_ticket = pd.NamedAgg(column='total' , aggfunc='mean'),
        avg_lead_time = pd.NamedAgg(column='lead_time' , aggfunc='mean'),
        orders_w_coupon = pd.NamedAgg(column='discount_id' , aggfunc='nunique'),
        suppliers_available = pd.NamedAgg(column='supplier_id',aggfunc='nunique'),).reset_index()    
    vars()[countries_KO_metrics[c]]['discount/gmv'] = vars()[countries_KO_metrics[c]]['discounts'].astype(float)/vars()[countries_KO_metrics[c]]['gmv'].astype(float)
    vars()[countries_KO_metrics[c]]['orders_no_coupon'] = vars()[countries_KO_metrics[c]]['supplier_orders'] - vars()[countries_KO_metrics[c]]['orders_w_coupon']
        

In [98]:
for c in group_metrics_countries:
    if c == 'ke':
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]][['year','month','week','week_start','KO_products','country_id','city','state_name','zone',
                                    'zone_mlolongo','supplier','supplier_id','gmv','gmv_usd','discounts','orders',
                                    'supplier_orders','canceled_supplier_orders','deliv_supplier_orders',
                                    'pending_orders','stores_w_purchase','avg_ticket','avg_lead_time',
                                    'orders_w_coupon','suppliers_available','discount/gmv','orders_no_coupon',
                                    ]]
    else:
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]][['year','month','week','week_start','KO_products','country_id','city','state_name','zone'
                                    ,'supplier','supplier_id','gmv','gmv_usd','discounts','orders',
                                    'supplier_orders','canceled_supplier_orders','deliv_supplier_orders',
                                    'pending_orders','stores_w_purchase','avg_ticket','avg_lead_time',
                                    'orders_w_coupon','suppliers_available','discount/gmv','orders_no_coupon',
                                    ]]
        

In [99]:
#juntamos las métricas de KO con las métricas generales
for c in group_metrics_countries:
    if c=='ke':
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].merge(vars()[sup_orders_items['ke']], how = 'left', on =['country_id',
                                    'state_name','city','zone','zone_mlolongo','year','month','supplier','supplier_id',
                                    'week','week_start','KO_products'])
        
    else:
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].merge(vars()[sup_orders_items[c]], how = 'left', on =['country_id',
                                    'state_name','city','zone','year','month','supplier','supplier_id',
                                    'week','week_start','KO_products'])
        

#### Register and new register stores - KO Metrics

In [100]:
registered_stores_KO_Metrics=pd.DataFrame()
registered_stores_KO_Metrics=customers3[customers3['country_id'].isin(group_metrics_countries)].reset_index(drop=True)
registered_stores_KO_Metrics['year'] = registered_stores_KO_Metrics['created_at'].map(extract_year)
registered_stores_KO_Metrics['week'] = registered_stores_KO_Metrics['created_at'].map(week_number)
registered_stores_KO_Metrics['month'] = registered_stores_KO_Metrics['created_at'].apply(lambda x: (pd.Timestamp(x)).date().strftime("%m"))
registered_stores_KO_Metrics['week_start'] =registered_stores_KO_Metrics.apply(lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)

In [101]:
countries_registered_stores= {}
for c in group_metrics_countries:
    countries_registered_stores[c] = c+'_registered_stores_KO_Metrics'
print(countries_registered_stores)

{'ke': 'ke_registered_stores_KO_Metrics', 'ma': 'ma_registered_stores_KO_Metrics'}


In [102]:
for c in countries_registered_stores:
    vars()[countries_registered_stores[c]] = registered_stores_KO_Metrics[registered_stores_KO_Metrics['country_id'] == c].reset_index(drop=True)

In [103]:
for c in countries_registered_stores:
    if 'ke' in c:
        vars()[countries_registered_stores[c]] = vars()[countries_registered_stores[c]][vars()[
                countries_registered_stores[c]].customer_status != 'REJECTED']
              
        vars()[countries_registered_stores[c]] = maps_2b.zonify(
            vars()[countries_registered_stores[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
        
        vars()[countries_registered_stores[c]] = vars()[countries_registered_stores[c]].drop(columns={'geometry'})
    else:
        pass

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.678855


In [104]:
for c in countries_registered_stores:
    if 'ke' in c:
            vars()[countries_registered_stores[c]] = vars()[countries_registered_stores[c]].groupby(['country_id','city','zone', 'zone_mlolongo']).agg(
            registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()
            
    else:
        vars()[countries_registered_stores[c]] = vars()[countries_registered_stores[c]].groupby(['country_id','city','zone']).agg(
        registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()

In [105]:
query_customers_ko_metrics = '''
select
country_id
, customer_id 
, c.name
, created_at
, legal_id
, state_name iso_code
, c.phone
, lat
, lon
, formatted_address
, customer_status
, ct.name cutomer_type
, c.owner_id
, c.additional_info
from wabi2bCustomers.customers c 
inner join wabi2bCustomers.customers_type ct 
on ct.type_id = c.customer_type_code
where c.country_id in '''+ wabi.stringify(group_metrics_countries)+'''
and created_at between "'''+tscope_start_str+'''" and "'''+tscope_end_str+'''"'''

In [106]:
customers_special_metric_df = sql_wabilytics.querySQL(query_customers_ko_metrics,schema = 'wabi2bCustomers')

In [107]:
customers_special_metric = customers_special_metric_df.copy(deep = True)

In [108]:
customers_special_metric['year'] = customers_special_metric['created_at'].map(extract_year)
customers_special_metric['week'] = customers_special_metric['created_at'].map(week_number)
customers_special_metric['month'] = customers_special_metric['created_at'].apply(lambda x: (pd.Timestamp(x)).date().strftime("%m"))
customers_special_metric['week_start'] =customers_special_metric.apply(lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)

In [109]:
customers_special_metric = maps_2b.zonify(customers_special_metric.reset_index(drop=True) ,zonas=full_zones).reset_index(drop=True)
customers_special_metric.rename(columns= {'zona':'city'}, inplace=True)
customers_special_metric = customers_special_metric.drop(columns={'geometry'})
customers_special_metric = maps_2b.zonify(customers_special_metric,maps_2b.zonas,column = 'zone')
customers_special_metric = customers_special_metric.drop(columns={'geometry'}).reset_index(drop=True)

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.218312


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

...

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:10.183728


In [110]:
new_registered_stores_KO_metrics_c = {}
for c in group_metrics_countries:
    new_registered_stores_KO_metrics_c[c] = c+'_new_registered_stores_KO_metrics'

print(new_registered_stores_KO_metrics_c)

for c in new_registered_stores_KO_metrics_c:
    vars()[new_registered_stores_KO_metrics_c[c]] = customers_special_metric[customers_special_metric['country_id'] == c].reset_index(drop=True)
for c in new_registered_stores_KO_metrics_c:
    if 'ke' in c:
        vars()[new_registered_stores_KO_metrics_c[c]] = vars()[new_registered_stores_KO_metrics_c[c]][vars()[
                new_registered_stores_KO_metrics_c[c]].customer_status != 'REJECTED']
              
        vars()[new_registered_stores_KO_metrics_c[c]] = maps_2b.zonify(
            vars()[new_registered_stores_KO_metrics_c[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
        
        vars()[new_registered_stores_KO_metrics_c[c]] = vars()[new_registered_stores_KO_metrics_c[c]].drop(columns={'geometry'})
    else:
        pass

{'ke': 'ke_new_registered_stores_KO_metrics', 'ma': 'ma_new_registered_stores_KO_metrics'}
Zonify time: 0:00:00.027000


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

In [111]:
for c in new_registered_stores_KO_metrics_c:
    if 'ke' in c:
            vars()[new_registered_stores_KO_metrics_c[c]] = vars()[new_registered_stores_KO_metrics_c[c]].groupby(['year','month','week', 'week_start' ,'country_id','city','zone', 'zone_mlolongo']).agg(
            new_registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()
            
    else:
        vars()[new_registered_stores_KO_metrics_c[c]] = vars()[new_registered_stores_KO_metrics_c[c]].groupby(['year','month','week', 'week_start' ,'country_id','city','zone']).agg(
        new_registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()

In [112]:
full_KO_metrics_registered = {}
for c in group_metrics_countries:
    full_KO_metrics_registered[c] = c+'_full_KO_metrics_registered_df'
    print(full_KO_metrics_registered)

{'ke': 'ke_full_KO_metrics_registered_df'}
{'ke': 'ke_full_KO_metrics_registered_df', 'ma': 'ma_full_KO_metrics_registered_df'}


In [113]:
for c in full_KO_metrics_registered:
    if 'ke' in c:
        vars()[full_KO_metrics_registered[c]] = vars()[new_registered_stores_KO_metrics_c[c]].merge(vars()[countries_registered_stores[c]], how='left', on=['country_id','city','zone', 'zone_mlolongo'])
    else:
        vars()[full_KO_metrics_registered[c]] = vars()[new_registered_stores_KO_metrics_c[c]].merge(vars()[countries_registered_stores[c]], how='left', on=['country_id','city','zone'])

In [114]:
#ponemos todas las cols del mismo tipo
for c in group_metrics_countries:
    vars()[full_KO_metrics_registered[c]]['year'] = vars()[full_KO_metrics_registered[c]]['year'].astype(np.int64)
    vars()[countries_KO_metrics[c]]['year'] = vars()[countries_KO_metrics[c]]['year'].astype(np.int64)
    
    vars()[countries_KO_metrics[c]]['month'] = vars()[countries_KO_metrics[c]]['month'].astype(np.int64)
    vars()[full_KO_metrics_registered[c]]['month'] = vars()[full_KO_metrics_registered[c]]['month'].astype(np.int64)
    
    vars()[countries_KO_metrics[c]]['week'] = vars()[countries_KO_metrics[c]]['week'].astype(np.int64)
    vars()[full_KO_metrics_registered[c]]['week'] = vars()[full_KO_metrics_registered[c]]['week'].astype(np.int64)

In [115]:
for c in group_metrics_countries:
    if 'ke' in c:
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].merge(vars()[full_KO_metrics_registered[c]], how='left', on=['year','month','week', 'week_start' ,'country_id','city','zone', 'zone_mlolongo'])
    else:
        vars()[countries_KO_metrics[c]] = vars()[countries_KO_metrics[c]].merge(vars()[full_KO_metrics_registered[c]], how='left', on=['year','month','week', 'week_start' ,'country_id','city','zone'])

## Suppliers Available Metric

In [116]:
supplier_areas_query = '''select sa.supplier_id,s.name supplier_name, s.country_id, ST_AsText(polygon) polygon
from suppliers_areas sa
inner join suppliers s on s.supplierId = sa.supplier_id
where country_id in '''+wabi.stringify(countries)+'''
'''
supplier_coordinates = sql_wabilytics.querySQL(supplier_areas_query, 'wabi2b')

In [117]:
supplier_coordinates["polygon"] = supplier_coordinates["polygon"].astype(str)
supplier_coordinates['polygon'] = supplier_coordinates['polygon'].apply(wkt.loads)
supplier_coordinates['polygon'] = supplier_coordinates['polygon'].apply(lambda x: Polygon(x))

In [118]:
## get supplier cities
supplier_coordinates['key'] = 1
full_zones['key'] = 1
supplier_coordinates_zones = supplier_coordinates.merge(full_zones, how='inner', on =['key'])
supplier_coordinates_zones = supplier_coordinates_zones[supplier_coordinates_zones.apply(lambda x: x['polygon'].intersects(x['geometry']), axis=1) == True].reset_index(drop=True)
supplier_coordinates_zones = supplier_coordinates_zones[['supplier_id', 'country_id','polygon', 'Name','key']].rename(columns = {'Name':'city'}).reset_index(drop=True)
supplier_coordinates_zones=supplier_coordinates_zones.drop(columns=['city'])

In [119]:
supplier_coordinates_zones1=pd.DataFrame()
countryes1=['ke', 'eg', 'ma']
supplier_coordinates_zones1=supplier_coordinates_zones[supplier_coordinates_zones["country_id"].isin(countryes1)].reset_index(drop=True)

In [120]:
maps_2b.zonas['key'] = 1
supplier_coordinates_citiess = supplier_coordinates_zones1.merge(maps_2b.zonas, how='inner', on =['key', 'country_id'])
supplier_coordinates_citiess = supplier_coordinates_citiess[supplier_coordinates_citiess.apply(lambda x: x['polygon'].intersects(x['geometry']), axis=1) == True].reset_index(drop=True)
supplier_coordinates_citiess["intersection_area"] = supplier_coordinates_citiess.apply(lambda x: x['polygon'].intersection(x['geometry']).area, axis=1)
supplier_coordinates_citiess["geometry_area"]=supplier_coordinates_citiess.apply(lambda x: x["geometry"].area, axis=1)
supplier_coordinates_citiess["to_keep"]=supplier_coordinates_citiess.apply(lambda x: True if (x["intersection_area"]/x["geometry_area"])>=0.8 else False, axis=1)
supplier_coordinates_citiess=supplier_coordinates_citiess[supplier_coordinates_citiess["to_keep"]==True].reset_index(drop=True)
supplier_zones1 = supplier_coordinates_citiess[['supplier_id', 'country_id', 'city','Name']].rename(columns = {'Name':'zone'}).reset_index(drop=True)

supplier_zones1 = supplier_zones1.drop_duplicates().reset_index(drop=True)

In [121]:
supplier_zones=supplier_zones1.copy()
supplier_zones.supplier_id = supplier_zones.supplier_id.astype(str)
suppliers_zones_g = supplier_zones.groupby(['country_id','city','zone']).agg(
    suppliers_available = pd.NamedAgg(column='supplier_id',aggfunc='nunique')).reset_index()
for c in countries_metrics_1:
    vars()[countries_metrics_1[c]] = vars()[countries_metrics_1[c]].merge(suppliers_zones_g, how='left', on=['country_id','city','zone'])

### Register and new register

In [122]:
registered_stores_df=pd.DataFrame()
registered_stores_df=customers3[customers3['country_id'].isin(countryes1)].reset_index(drop=True)
registered_stores_df['year'] = registered_stores_df['created_at'].map(extract_year)
registered_stores_df['week'] = registered_stores_df['created_at'].map(week_number)
registered_stores_df['month'] = registered_stores_df['created_at'].apply(lambda x: (pd.Timestamp(x)).date().strftime("%m"))
registered_stores_df['week_start'] =registered_stores_df.apply(lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)

In [123]:
countryes_registered_stores = {}
for c in countryes1:
    countryes_registered_stores[c] = c+'_registered_stores_df'

print(countryes_registered_stores)

for c in countryes_registered_stores:
    vars()[countryes_registered_stores[c]] = registered_stores_df[registered_stores_df['country_id'] == c].reset_index(drop=True)
for c in countryes_registered_stores:
    if 'ke' in c:
        vars()[countryes_registered_stores[c]] = vars()[countryes_registered_stores[c]][vars()[
                countryes_registered_stores[c]].customer_status != 'REJECTED']
              
        vars()[countryes_registered_stores[c]] = maps_2b.zonify(
            vars()[countryes_registered_stores[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
        
        vars()[countryes_registered_stores[c]] = vars()[countryes_registered_stores[c]].drop(columns={'geometry'})
    else:
        pass

{'ke': 'ke_registered_stores_df', 'eg': 'eg_registered_stores_df', 'ma': 'ma_registered_stores_df'}


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.618808


In [124]:
for c in countryes_registered_stores:
    if 'ke' in c:
            vars()[countryes_registered_stores[c]] = vars()[countryes_registered_stores[c]].groupby(['country_id','city','zone', 'zone_mlolongo']).agg(
            registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()
            
    else:
        vars()[countryes_registered_stores[c]] = vars()[countryes_registered_stores[c]].groupby(['country_id','city','zone']).agg(
        registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()

In [125]:
query_customers5 = '''select
country_id
, customer_id 
, c.name
, created_at
, legal_id
, state_name iso_code
, c.phone
, lat
, lon
, formatted_address
, customer_status
, ct.name cutomer_type
, c.owner_id
, c.additional_info
from wabi2bCustomers.customers c 
inner join wabi2bCustomers.customers_type ct 
on ct.type_id = c.customer_type_code
where c.country_id in '''+ wabi.stringify(group_metrics_countries)+'''
and created_at between "'''+tscope_start_str+'''" and "'''+tscope_end_str+'''"'''

In [126]:
customers_metric_sepcial = sql_wabilytics.querySQL(query_customers5,schema = 'wabi2bCustomers')

In [127]:
customers_metric_sepcial['year'] = customers_metric_sepcial['created_at'].map(extract_year)
customers_metric_sepcial['week'] = customers_metric_sepcial['created_at'].map(week_number)
customers_metric_sepcial['month'] = customers_metric_sepcial['created_at'].apply(lambda x: (pd.Timestamp(x)).date().strftime("%m"))
customers_metric_sepcial['week_start'] =customers_metric_sepcial.apply(lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)

In [128]:
customers_metric_sepcial = maps_2b.zonify(customers_metric_sepcial.reset_index(drop=True) ,zonas=full_zones).reset_index(drop=True)
customers_metric_sepcial.rename(columns= {'zona':'city'}, inplace=True)
customers_metric_sepcial = customers_metric_sepcial.drop(columns={'geometry'})
customers_metric_sepcial = maps_2b.zonify(customers_metric_sepcial,maps_2b.zonas,column = 'zone')
customers_metric_sepcial = customers_metric_sepcial.drop(columns={'geometry'}).reset_index(drop=True)

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:00.164642


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

...

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

Zonify time: 0:00:09.069951


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

In [129]:
new_registered_stores_c = {}
for c in countryes1:
    new_registered_stores_c[c] = c+'_new_registered_stores_df'

print(new_registered_stores_c)

for c in new_registered_stores_c:
    vars()[new_registered_stores_c[c]] = customers_metric_sepcial[customers_metric_sepcial['country_id'] == c].reset_index(drop=True)
for c in new_registered_stores_c:
    if 'ke' in c:
        vars()[new_registered_stores_c[c]] = vars()[new_registered_stores_c[c]][vars()[
                new_registered_stores_c[c]].customer_status != 'REJECTED']
              
        vars()[new_registered_stores_c[c]] = maps_2b.zonify(
            vars()[new_registered_stores_c[c]],
            zonas_mlolongo,column='zone_mlolongo').reset_index(drop=True)
        
        vars()[new_registered_stores_c[c]] = vars()[new_registered_stores_c[c]].drop(columns={'geometry'})
    else:
        pass

{'ke': 'ke_new_registered_stores_df', 'eg': 'eg_new_registered_stores_df', 'ma': 'ma_new_registered_stores_df'}
Zonify time: 0:00:00.032374


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1428: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\li

In [130]:
for c in new_registered_stores_c:
    if 'ke' in c:
            vars()[new_registered_stores_c[c]] = vars()[new_registered_stores_c[c]].groupby(['year','month','week', 'week_start' ,'country_id','city','zone', 'zone_mlolongo']).agg(
            new_registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()
            
    else:
        vars()[new_registered_stores_c[c]] = vars()[new_registered_stores_c[c]].groupby(['year','month','week', 'week_start' ,'country_id','city','zone']).agg(
        new_registered_stores = pd.NamedAgg(column='customer_id',aggfunc='nunique')).reset_index()

In [131]:
full_metrics_registered = {}
for c in countryes1:
    full_metrics_registered[c] = c+'_full_metrics_registered_df'

print(full_metrics_registered)
for c in full_metrics_registered:
    if 'ke' in c:
        vars()[full_metrics_registered[c]] = vars()[new_registered_stores_c[c]].merge(vars()[countryes_registered_stores[c]], how='left', on=['country_id','city','zone', 'zone_mlolongo'])
    else:
        vars()[full_metrics_registered[c]] = vars()[new_registered_stores_c[c]].merge(vars()[countryes_registered_stores[c]], how='left', on=['country_id','city','zone'])

{'ke': 'ke_full_metrics_registered_df', 'eg': 'eg_full_metrics_registered_df', 'ma': 'ma_full_metrics_registered_df'}


In [132]:
for c in countryes1:
    if 'ke' in c:
        vars()[countries_metrics_1[c]] = vars()[countries_metrics_1[c]].merge(vars()[full_metrics_registered[c]], how='left', on=['year','month','week', 'week_start' ,'country_id','city','zone', 'zone_mlolongo'])
    else:
        try:
            vars()[countries_metrics_1[c]] = vars()[countries_metrics_1[c]].merge(vars()[full_metrics_registered[c]], how='left', on=['year','month','week', 'week_start' ,'country_id','city','zone'])
        except:
            pass
for c in countries_metrics_1:
    vars()[countries_metrics_1[c]]=vars()[countries_metrics_1[c]].fillna(0)

## LETS PASTE THINGS

### Customer info

In [ ]:
for c,f in countries_files.items():
    print(c)
    print(f)

In [ ]:
for c,f in countries_files.items():
    supp_id = 0
    try:
        c,supp_id = c.split('_')[0], c.split('_')[1]
    except:
        pass
    file = gSheets.readSheetByKey(f)
    stores_info = file.worksheet('store_info')
    vars()[countries_customers[c]] = vars()[countries_customers[c]].sort_values(
        'created_at', ascending = True).reset_index(drop=True)
    if stores_paste_boolean == True:
        stores_info.clear()
        set_with_dataframe(stores_info, vars()[countries_customers[c]], row=1, col=1, include_index=False,
                       include_column_header=True, resize=False)
        print(f'Store information was updated for {c}{supp_id} in the file with the key: {f}')
    else:
        print(f'Store information was NOT updated for {c} in the file with the key: {f}. Change the parameter stores_paste_boolean to True to update the file.')

### Products

### Metrics

In [133]:
now = datetime.datetime.utcnow() 
t_w=now.isocalendar()[1]
l_w=t_w -1
for c,f in countries_files.items():
    supp_id = 0
    try:
        c,supp_id = c.split('_')[0], c.split('_')[1]
    except:
        pass
    if len(c+str(supp_id)) == 3:
        file = gSheets.readSheetByKey(f)  
        metrics = file.worksheet('metrics')
        metrics_original = pd.DataFrame(metrics.get_all_records())
        vars()[countries_metrics_1[c]].supplier_id = vars()[countries_metrics_1[c]].supplier_id.astype(int)
        print('check if there are records in sheet. Only new values are being pasted')
        metrics_to_paste = vars()[countries_metrics_1[c]].copy(deep=True)
        metrics_to_paste = metrics_to_paste.sort_values(
            ['week_start','supplier_id'], ascending = [True,True]).reset_index(drop=True)
        if metrics_paste_boolean == True:
            if now.weekday()==0:
                metrics_to_paste=metrics_to_paste[(metrics_to_paste['week']==str(t_w))|(metrics_to_paste['week']==str(l_w))].reset_index(drop=True)
                correct_metrics=metrics_original.append(metrics_to_paste).reset_index(drop=True)
                correct_metrics=correct_metrics.drop_duplicates(["week","week_start", "country_id", "city", "state_name", "zone", "supplier_name", "supplier_id"]).reset_index(drop=True)
                metrics.resize(rows = 2)
                set_with_dataframe(metrics, correct_metrics, row=1, col=1,
                                include_index=False, include_column_header=True, resize=False)
                print(f'Store information was updated for {c} in the file with the key: {f}')
            else:
                metrics_to_paste=metrics_to_paste[metrics_to_paste['week']==now.isocalendar()[1]].reset_index(drop=True)
                correct_metrics=metrics_original.append(metrics_to_paste).reset_index(drop=True)
                correct_metrics=correct_metrics.drop_duplicates(["week","week_start", "country_id", "city", "state_name", "zone", "supplier_name", "supplier_id"]).reset_index(drop=True)
                metrics.resize(rows = 2)
                set_with_dataframe(metrics, correct_metrics, row=1, col=1,
                                include_index=False, include_column_header=True, resize=False)
                print(f'Store information was updated for {c} in the file with the key: {f}')
        else:
            print(f'Store information was NOT updated for {c} in the file with the key: {f}. Change the parameter products_paste_boolean to True to update the file.')
    else:
        pass

check if there are records in sheet. Only new values are being pasted


C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\2430877115.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  correct_metrics=metrics_original.append(metrics_to_paste).reset_index(drop=True)


Store information was updated for ke in the file with the key: 1GHVsvDAIR3sY5ps4YbNilY_V-MJleLteG8-0LymHnkE
check if there are records in sheet. Only new values are being pasted


C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\2430877115.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  correct_metrics=metrics_original.append(metrics_to_paste).reset_index(drop=True)


Store information was updated for ma in the file with the key: 1ED1amF3VN23LjZYh2kdMISTSPhD_ySsYdfAQRF006RM


### Daily Orders

In [134]:
for c,f in countries_files.items():
    print(c)
    #default state == 0 >> so there won't be any filtering of supplier id
    supp_id = 0
    try:
        c,supp_id = c.split('_')[0], c.split('_')[1]
    except:
        pass

    file = gSheets.readSheetByKey(f)
    daily_orders = file.worksheet('daily_orders')
    
    daily_orders_original = pd.DataFrame(daily_orders.get_all_records())
    try:
        daily_orders_original.created_at = pd.to_datetime(daily_orders_original.created_at)
        daily_orders_original['supplier_order_id'] = daily_orders_original['supplier_order_id'].astype(str)
    except:
        print('0: check if there are records in sheet')
    print('1:check if there are records in sheet. Only new values are being pasted')
    daily_orders_to_paste = vars()[countries_orders[c]].copy(deep=True)
    
    daily_orders_to_paste.created_at = pd.to_datetime(daily_orders_to_paste.created_at)
    if supp_id != 0:
        daily_orders_to_paste = daily_orders_to_paste[daily_orders_to_paste.supplier_id == supp_id].reset_index(drop=True)
    daily_orders_to_paste = daily_orders_to_paste.sort_values(
        'created_at', ascending = True).reset_index(drop=True)
    if orders_paste_boolean == True:
        daily_orders_to_paste["supplier_order_id"]=daily_orders_to_paste["supplier_order_id"].astype(str)
        daily_orders_correct=daily_orders_original.append(daily_orders_to_paste).reset_index(drop=True)
        daily_orders_correct["supplier_order_id"]=daily_orders_correct["supplier_order_id"].astype(str)
        daily_orders_correct=daily_orders_correct.drop_duplicates(["supplier_order_id"], keep="last").reset_index(drop=True)
        daily_orders_correct = daily_orders_correct.sort_values(
        'created_at', ascending = True).reset_index(drop=True)
        daily_orders.resize(rows = 2)
        set_with_dataframe(daily_orders, daily_orders_correct, row=1, col=1, 
                     include_index=False, include_column_header=True, resize=False)
        print(f'Order information was updated for {c}{supp_id} in the file with the key: {f}\n\n')
        print(f''+str(len(daily_orders_correct)-len(daily_orders_original))+' rows where added')
    else:
        print(f'Order information was NOT updated for {c}{supp_id} in the file with the key: {f}. Change the parameter orders_paste_boolean to True to update the file.\n\n')
        
    time.sleep(60)

ke
1:check if there are records in sheet. Only new values are being pasted


C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\2334023813.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_orders_correct=daily_orders_original.append(daily_orders_to_paste).reset_index(drop=True)


Order information was updated for ke0 in the file with the key: 1GHVsvDAIR3sY5ps4YbNilY_V-MJleLteG8-0LymHnkE


0 rows where added
ma
1:check if there are records in sheet. Only new values are being pasted


C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\2334023813.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  daily_orders_correct=daily_orders_original.append(daily_orders_to_paste).reset_index(drop=True)


Order information was updated for ma0 in the file with the key: 1ED1amF3VN23LjZYh2kdMISTSPhD_ySsYdfAQRF006RM


0 rows where added


### KO Metrics

In [135]:
KO_countries_metrics=pd.concat([ke_KO_metrics], axis=0).reset_index(drop=True)

In [136]:
file = gSheets.readSheetByKey('1OFnf-w-svGhFLNX0YGpv-KYdEVtCdbqLIBBLpf-u9h8')
sheet_tab = file.worksheet('Metrics KO - NO KO')
df = pd.DataFrame(sheet_tab.get_all_records()).reset_index(drop=True)

In [137]:
updated_df = KO_countries_metrics
if len(df) > 0:
    df_without_4_weeks = df.loc[~df['week_start'].isin(weeks_list)].reset_index(drop=True)
    #filtro y me quedo con las semanas menores a las 4 semanas calculadas
    updated_df = updated_df.append(df_without_4_weeks)
updated_df.sort_values(by =["year", "month", "week", "country_id","supplier_id","supplier"], inplace = True)
sheet_tab.clear()
set_with_dataframe(sheet_tab, updated_df)
    

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_1820\499319786.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = updated_df.append(df_without_4_weeks)
